In [6]:
!pip install git+https://github.com/twintproject/twint
!pip install aiohttp_socks

  Cloning https://github.com/twintproject/twint to /private/var/folders/8q/h7x35g6s20l48s4yxqvzw1w40000gp/T/pip-req-build-1sfehml9
  Running setup.py bdist_wheel for twint ... done
  Stored in directory: /private/var/folders/8q/h7x35g6s20l48s4yxqvzw1w40000gp/T/pip-ephem-wheel-cache-ec49en82/wheels/44/02/e0/86b03deb4fb12efcb200d22b87efb9b38ee14ed6bfaacd02d2
Successfully built twint


In [7]:
#!pip install -q tweepy
%reload_ext autoreload
%autoreload 2
import twint
import logging
import pandas as pd
import datetime
import tweepy
import json
from utils.extract import db_connection, download_data
logging.getLogger().setLevel(logging.WARNING)

In [8]:
#credenciales de db, twitter y emisoras
with open('creds.txt', encoding='utf-8') as data_file:
        creds = json.loads(data_file.read())

with open('tw_creds.txt', encoding='utf-8') as data_file:
    tw_creds = json.loads(data_file.read())

lista_cuentas = pd.read_csv('Cuentas BMBV.csv')



In [9]:
c = twint.Config()
c.Username = "BWcapitalMex"
c.Limit = 10
c.Pandas = True

In [ ]:
twint.run.Search(c)

In [11]:
tweets_df = twint.storage.panda.Tweets_df

In [ ]:
tweets_df.head()

In [13]:
def obtiene_todos_tweets(cuenta):
    
    c = twint.Config()
    c.Username = cuenta.replace('@','') # por si metemos la cuenta con @
    c.Pandas = True
  
    twint.run.Search(c)
  
    df = twint.storage.panda.Tweets_df
  
    return df

def obtiene_todo_contenido(contenido, limit=1000):
    
    c = twint.Config()
    c.Search = contenido
    c.Pandas = True
    c.Limit = limit
  
    twint.run.Search(c)
  
    df = twint.storage.panda.Tweets_df
  
    return df
  
def update_in_bulk_db(creds):
    """
    Añade todos los tweets de una vez
    """
    query_table = """
            CREATE TABLE IF NOT EXISTS
                tweets (
                    id integer not null,
                    user_id integer,
                    date text not null,
                    time text not null,
                    timezone text not null,
                    location text not null,
                    user text not null,
                    tweet text not null,
                    replies integer,
                    likes integer,
                    retweets integer,
                    hashtags text,
                    link text,
                    retweet bool,
                    user_rt text,
                    mentions text,
                    date_update text not null,
                    PRIMARY KEY (id)
                );
        """
    conn = db_connection(creds)
    
    try:
        download_data(conn,query_table)
    except Exception as e:
        print('Error al crear tabla de tweets'.format(e))
    
    
    
    try:
      
      for cuenta in lista_cuentas['Cuentas']:
        df = obtiene_todos_tweets(cuenta)
        
        conn = db_connection(creds)
        date_time = str(datetime.now())
      
        lista_tweets = df.values.tolist()
        lista_tweets = [tuple(tweet.append(date_time)) for tweet in lista_tweets]
        
    
        print('se van a guardar {} tweets de la cuenta: {}'.format(len(lista_tweets), lista_tweets[0][12]))
    
        data_ready = str(lista_tweets[1:-1])
    
        query= """INSERT INTO tweets VALUES {};""".format(data_ready)
    
        download_data(conn,query)
      
    except Exception as e:
        print('Error al insertar en la base'.format(e))


In [14]:
update_in_bulk_db(creds)

ERROR:root:Error en download data
ERROR:root:syntax error at or near "user"
LINE 10:                     user text not null,
                             ^



Error al insertar en la base
1046172818684153856 2018-09-29 18:00:11 CDT <BMVMercados> Traxion, empresa dedicada la industria del autotransporte y logística en México celebra su primer año como #EmpresaBolsa. pic.twitter.com/4ajFvGSJnE
1046127520309886976 2018-09-29 15:00:11 CDT <BMVMercados> .@elcidresorts controladora de acciones de empresas hoteleras y de esparcimiento, cumple 21 años de listado en #LaBolsaDeMéxico. #EmpresasBolsa pic.twitter.com/FuqCwpdAyl
1046082244413214726 2018-09-29 12:00:16 CDT <BMVMercados> Conoce el comportamiento del mercado en nuestro boletín semanal pic.twitter.com/aaKCvcMvSQ
1045848135845761030 2018-09-28 20:30:01 CDT <BMVMercados> Grupo México Transportes cumple un año como emisora de #LaBolsaDeMéxico. ¡Felicidades! pic.twitter.com/ZeYkacoYzr
1045840599159640064 2018-09-28 20:00:04 CDT <BMVMercados> .@ASURMexico ofrece servicios aeroportuarios, hoy celebra 18 años de listado en #LaBolsaDeMéxico. ¡Muchas felicidades!. #EmpresasBolsa pic.twitter.com/0GDHH

1044738360366047234 2018-09-25 19:00:10 CDT <BMVMercados> .@jbonham_carter, enviada especial para comercio del Reino Unido en México, sostuvo un encuentro con directivos de #LaBolsaDeMéxico y @mexico2_pmc para profundizar nuestra colaboración sobre #bonosverdes, infraestructura sustentable y mejores prácticas en #financiamientoclimatico pic.twitter.com/6jpbgLMmxY
1044684254662283264 2018-09-25 15:25:10 CDT <BMVMercados> Cierre de Mercado: $BMV S&P/BMV IPC 49,651.55 / +0.47% Dólar Spot: 18.9970
1044683519371423750 2018-09-25 15:22:14 CDT <BMVMercados> $MexDer Precios de liquidación $FUTUROS S&P/BMV IPC DC18= 50,440 / +323.00 / -0.64% USD/MX DC18= $19.2560 / +0.05 / +0.25%
1044670416881700864 2018-09-25 14:30:11 CDT <BMVMercados> Hoy se conocieron los datos del IGAE en nuestro país, ¿Cuál fue la reacción de los inversionistas? hoy en #DesdeLaBolsa Carlos González Tabares Director de Análisis y Estrategia Bursátil Monex14:40hrs FOROtv
1044608285742174211 2018-09-25 10:23:17 CDT <BMVMercad

1043213466390147083 2018-09-21 14:00:47 CDT <BMVMercados> .@FitchRatings le asignó a la Bolsa Mexicana de valores, que cotiza bajo la clave “BOLSA”, las calificaciones en escala internacional (IDRs) de largo plazo en moneda extranjera y local de 'BBB+' y de largo plazo en escala nacional 'AAA(mex)', con Perspectiva Estable. pic.twitter.com/mNRODUrr37
1043135848663994369 2018-09-21 08:52:21 CDT <BMVMercados> Apertura de Mercado: $BMV S&P/BMV IPC 49,436.63 / -0.15% Dólar Spot: 18.8885
1043135818066526209 2018-09-21 08:52:14 CDT <BMVMercados> $MexDer Apertura $FUTUROS S&P/BMV IPC DC18= 50,405 / +112.00 / +0.22% USD/MX DC18= $19.1523 / +0.08 / +0.44%
1042952308076765184 2018-09-20 20:43:02 CDT <BMVMercados> Estamos impulsando la Educación Financiera con el lanzamiento del blog #HablemosDeBolsa y el rediseño de la #EscuelaBolsaMexicana de la que han egresado más de 20 mil estudiantes y hemos recibido a 25 mil personas que visitan la sede de #LaBolsaDeMéxico pic.twitter.com/egXUZAr8ES
104295

1041685708266713088 2018-09-17 08:50:01 CDT <BMVMercados> $MexDer Apertura $FUTUROS S&P/BMV IPC SP18= 49,720 / -77.00 / -0.15% USD/MX SP18= $18.8384 / -0.04 / -0.19%
1041371159672115202 2018-09-16 12:00:06 CDT <BMVMercados> En #LaBolsaDeMéxico celebramos el orgullo de ser mexicanos. ¡Felices Fiestas Patrias! pic.twitter.com/LLXrKse3wN
1040701062578151424 2018-09-14 15:37:23 CDT <BMVMercados> Cierre de Mercado: $BMV S&P/BMV IPC 49,611.93 / -0.16% Dólar Spot: 18.8623
1040700696000167937 2018-09-14 15:35:55 CDT <BMVMercados> $MexDer Precios de liquidación $FUTUROS S&P/BMV IPC SP18= 49,797 / -56.00 / -0.11% USD/MX SP18= $18.8748 / +0.01 / +0.03%
1040681590882480129 2018-09-14 14:20:00 CDT <BMVMercados> Factores podrían influenciar en el tipo de cambio hacia la siguiente semana hoy en #DesdeLaBolsa Fernando Bolaños Analista Bursátil Monex 14:40hrs FOROtv
1040657575048564736 2018-09-14 12:44:35 CDT <BMVMercados> Seguramente has escuchado hablar acerca de los Derivados Financieros, pero proba

...

766731722796904448 2016-08-19 15:21:02 CDT <BMVMercados> $BMV $IPC 48,297.46 / -139.64 / -0.29% / Dólar Spot: 18.2470
766731464369070080 2016-08-19 15:20:00 CDT <BMVMercados> $MexDer Precios de liquidación $FUTUROS $IPC SP16= 48,397/USD SP16=18.3192
766701264134479872 2016-08-19 13:20:00 CDT <BMVMercados> $BMV $IPC 48,308.10 / -129.00 / -0.27% / Dólar Spot: 18.2805
766692458759458817 2016-08-19 12:45:01 CDT <BMVMercados> Máximos históricos en EUA y México hoy en #DesdeLaBolsa Héctor Hernández Subdirector de Promoción Scotia 13:20hrs FOROtv
766681173129064448 2016-08-19 12:00:10 CDT <BMVMercados> ¿Sabes qué es el #MercadoGlobal de #LaBolsa ? una plataforma para comprar valores extranjeros sin salir del país  http://bit.ly/1hbaIzV 
766666069280821249 2016-08-19 11:00:09 CDT <BMVMercados> ¿Cómo funciona el piloto de Mercado de Carbono? pic.twitter.com/7JhThcW8mX
766650977952763904 2016-08-19 10:00:11 CDT <BMVMercados> .@bbva celebra su 10° aniversario de colocación de acciones en #LaBolsa

764185944312295424 2016-08-12 14:45:01 CDT <BMVMercados> En la COP 21, México se comprometió a reducir la emisión de gases de efecto invernadero 25%  para el 2030.
764182190007554048 2016-08-12 14:30:06 CDT <BMVMercados> En abril de 2012, el Congreso mexicano aprobó por unanimidad la Ley General de Cambio Climático (LGCC). pic.twitter.com/onTVQm6aQv
764178394619662336 2016-08-12 14:15:01 CDT <BMVMercados> Las empresas participantes de mercados de carbono una alternativa para compensar sus emisiones.
764174678491443201 2016-08-12 14:00:15 CDT <BMVMercados> Los bonos de carbono provienen de proyectos que reducen las emisiones de gases de efecto invernadero.
764164552271376384 2016-08-12 13:20:01 CDT <BMVMercados> $BMV $IPC 48,314.09 / +27.86 / +0.06% / Dólar Spot: 18.2840
764155744216166400 2016-08-12 12:45:01 CDT <BMVMercados> Política Monetaria del Banco de México hoy en #DesdeLaBolsa Araceli Espinosa Directora de Análisis de Deuda Actinver 13:20hrs FOROtv
764097055216181249 2016-08-12

761627836528537600 2016-08-05 13:20:01 CDT <BMVMercados> $BMV $IPC 47,029.63  / 88.17  / 0.19% / Dólar Spot: 18.8530
761619027311591424 2016-08-05 12:45:00 CDT <BMVMercados> Hoy analicemos #DesdeLaBolsa los precios del petróleo y del tipo de cambio con Juan Francisco Caudillo, Analista Monex 13:20hrs por FOROtv
761607737570500608 2016-08-05 12:00:09 CDT <BMVMercados> En #LaBolsa no sólo se financian grandes empresas, también medianas a través del Mercado de Deuda, escríbenos promocion@grupobmv.com.mx.
761601689648451584 2016-08-05 11:36:07 CDT <BMVMercados> El objetivo de #CKDs “FCICK” es invertir en proyectos futuros o en etapa de operación en México, conoce detalles en  http://bit.ly/2aO5fzc 
761601171244978178 2016-08-05 11:34:03 CDT <BMVMercados> La 1ra emisión fue por $700mdp a 13 años y representó 20% del monto máximo de emisión, con posibilidad de subsecuentes
761600688669327360 2016-08-05 11:32:08 CDT <BMVMercados> La emisión de #CKDs con clave “FCICK” se realizó por un monto m

758805427383480324 2016-07-28 18:24:46 CDT <BMVMercados> El 89% de los varones y 92% de las mujeres están de acuerdo con que el trabajo en el hogar deber ser compartido pic.twitter.com/khE3EGRpcg
758804606197477379 2016-07-28 18:21:30 CDT <BMVMercados> La Encuesta Amafore 2015, Ahorro y Futuro: Una perspectiva de género, señala que el 48% de las mujeres ahorran #JuevesDeBolsa
758803674101522433 2016-07-28 18:17:48 CDT <BMVMercados> En nuestro #JuevesDeBolsa Isabel Galván habla de la mujer como parte fundamental en la toma de decisiones. pic.twitter.com/wDI4cDoSmp
758801711070707712 2016-07-28 18:10:00 CDT <BMVMercados> Así arranca nuestro #JuevesDeBolsa con Isabel Galván Muñoz, gracias a todos por hacerlo posible. pic.twitter.com/D0RvCCHS3R
758799235512795136 2016-07-28 18:00:10 CDT <BMVMercados> #LaBolsa1erSemestre pic.twitter.com/KC2kicAEfY
758759181990060032 2016-07-28 15:21:00 CDT <BMVMercados> $BMV $IPC 46,883.87 / +71.80 / +0.15% / Dólar Spot: 18.8820
758758932462514177 2016-07-2

756504110220836864 2016-07-22 10:00:09 CDT <BMVMercados> Muchas felicidades a NEMAK por su 13° aniversario como emisora en #LaBolsa. #EmpresasBolsa
756496527758094336 2016-07-22 09:30:01 CDT <BMVMercados> El INEGI informa que en la primera quincena de julio de 2016 el INPC presentó un incremento de 0.28 por ciento.
756486651283550209 2016-07-22 08:50:47 CDT <BMVMercados> $BMV $IPC 47,455.23  / +90.42  / +0.19% / Dólar Spot: 18.5940
756486554634203137 2016-07-22 08:50:24 CDT <BMVMercados> $MexDer Apertura $FUTUROS $IPC SP16= 47,665/USD SP16=18.7125
756222468939845633 2016-07-21 15:21:01 CDT <BMVMercados> $BMV $IPC 47,364.81  / -140.44  / -0.30% / Dólar Spot: 18.5980
756222217407520768 2016-07-21 15:20:01 CDT <BMVMercados> $MexDer Precios de liquidación $FUTUROS $IPC SP16= 47,586/USD SP16=18.7524
756192019953758209 2016-07-21 13:20:01 CDT <BMVMercados> $BMV $IPC 47,489.84 / -15.41 / -0.03% / Dólar Spot: 18.6155
756183211030511616 2016-07-21 12:45:01 CDT <BMVMercados> Rally alcista del IP